## Mounting Google Drive

In [ ]:
#code to mount drive
from google.colab import drive
drive.mount('/content/drive/')

## Importing Dependencies

In [ ]:
import os
if not os.path.exists("helper_functions.py"):
    print("Downloading helper functions...")
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
    print("Helper functions file already exists, skipping download...")
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys, load_and_prep_image

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import accuracy_score
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

## Gathering Data

In [ ]:
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=False, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

## Accessing & Visualizing Data

In [ ]:
# Features of Food101 TFDS
ds_info.features

In [ ]:
# Get class names
class_names = ds_info.features["label"].names
train_one_sample = train_data.take(1)
train_one_sample

In [ ]:
# Output info about our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
  """)


  Image shape: (512, 512, 3)
  Image dtype: <dtype: 'uint8'>
  Target class from Food101 (tensor form): 56
  Class name (str form): huevos_rancheros
  


## Data Preprocessing

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

In [ ]:
# Preprocess a single sample image and check the outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"Shape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}")

Shape: (224, 224, 3),
Datatype: <dtype: 'float32'>


In [ ]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32)

In [ ]:
train_data, test_data

## Create modelling callbacks & Setting mixed precision

In [ ]:
from helper_functions import create_tensorboard_callback

checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      montior="val_accuracy", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=1) # don't print out whether or not model is being saved

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16") # set global policy to mixed precision
mixed_precision.global_policy()

## Build feature extraction model


In [ ]:
# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model
inputs = layers.Input(shape=input_shape, name="input_layer", dtype=tf.float16)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dropout(.3)(x)
x = layers.Dense(len(class_names))(x) # want one output neuron per class

# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
# Check out our model
model.summary()

In [ ]:
# Fit the model with callbacks
history_feature_extract = model.fit(train_data,
                                    epochs=5,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data,
                                    validation_steps=int(0.15 * len(test_data)),
                                    callbacks=[create_tensorboard_callback("training_logs",
                                                                           "efficientnetb0_101_classes_all_data_feature_extract"),
                                               model_checkpoint]
                                   )

In [ ]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

In [ ]:
!ls model_checkpoints/

In [ ]:
# Create save path to drive
save_dir = "drive/MyDrive/food_app/extractor/"
os.makedirs(save_dir) # Make directory if it doesn't exist

# Save model
model.save(save_dir)
model.save("/content/drive/MyDrive/food_app/ExtractorModel.hdf5")

In [ ]:
loaded_saved_model = tf.keras.models.load_model(save_dir)

## Preparing our model's layers for fine-tuning

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [ ]:
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [ ]:
# Compile the model
loaded_saved_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

In [ ]:
# Start to fine-tune (all layers)
history_fine_tuning = loaded_saved_model.fit(train_data,
                                             epochs=20, # fine-tune for a maximum of 100 epochs
                                             steps_per_epoch=len(train_data),
                                             validation_data=test_data,
                                             validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                             callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # track the model training logs
                                             model_checkpoint, # save only the best model during training
                                             early_stopping, # stop model after X epochs of no improvements
                                             reduce_lr]) # reduce the learning rate after X epochs of no improvements

In [ ]:
## Save model to Google Drive
loaded_saved_model.save("/content/drive/MyDrive/food_app/fine_tuning/")

In [ ]:
# Evaluate mixed precision trained loaded model
results_loaded_gs_model_fine_tuned = loaded_saved_model.evaluate(test_data)
results_loaded_gs_model_fine_tuned

In [ ]:
loaded_saved_model.save("/content/drive/MyDrive/food_app/FinalModel.hdf5")

In [ ]:
# Load model previously saved above
model = tf.keras.models.load_model("/content/drive/MyDrive/food_app/FinalModel.hdf5")

In [ ]:
# Evaluate mixed precision trained loaded model
results_loaded_model_fine_tuned = model.evaluate(test_data)
results_loaded_model_fine_tuned

## Model Evaluation and Visualization
#### SKlearn's Accuracy Score

In [ ]:
pred_probs = model.predict(test_data, verbose=1)
pred_classes = pred_probs.argmax(axis=1)
y_labels = []
for images, labels in test_data.unbatch():
    y_labels.append(labels.numpy())
sklearn_acc = accuracy_score(y_labels, pred_classes)
sklearn_acc

## Predicting User Input

In [ ]:
def pred_plot_custom(folder_path):
  import os

  custom_food_images = [folder_path + img_path for img_path in os.listdir(folder_path)]
  fig, a = plt.subplots(len(custom_food_images),2, figsize=(15, 5 * len(custom_food_images)))
  i = 0
  # Load and preprocess the image
  img = load_and_prep_image(img, scale=False)

  # Make predictions using the model
  pred_prob = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class with the highest probability
  pred_class_index = pred_prob.argmax()
  pred_class = class_names[pred_class_index]
  pred_accuracy = pred_prob[0, pred_class_index]

  # Now, pred_class contains the predicted class label, and pred_accuracy contains its corresponding probability.
  print(f"Image: {img}, Predicted Class: {pred_class}, Accuracy: {pred_accuracy:.2%}")

In [ ]:
# pred_plot_custom("/content/drive/MyDrive/food_app/images/food_img/")